# Merge

In [1]:
import pathlib
import sys
import pickle

from typing import List

import numpy as np
import pandas as pd

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# add the 'src' directory to path to import modules
src_dir = pathlib.Path().cwd().resolve().parent / 'src'
#src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)
# import my class code from the source
# %aimport src-dir.filename

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PROJECT_DIR = pathlib.Path.cwd().resolve().parent
#DATA_DIR = PROJECT_DIR / 'data'
# RAW_DATA_DIR = PROJECT_DIR / 'data' / '01-raw'
# CLEANED_DATA_DIR = PROJECT_DIR / 'data' / '02-cleaned'
STDZED_DATA_DIR = PROJECT_DIR / 'data' / '03-standardized'
# REF_DATA_DIR = PROJECT_DIR / 'data' / 'reference'
MERGED_DATA_DIR = PROJECT_DIR / 'data' / '04-merged'

## Select League Data

In [2]:
nations = ['germany', 'united-kingdom', 'spain', 'italy',
          'france', 'united-kingdom', 'germany', 'netherlands',
          'russian-federation', 'scotland', 'portugal', 'switzerland',
          'belgium', 'turkey', 'poland', 'united-kingdom']

# poland, switzerland are multileague

leagues = ['bundesliga', 'english-premier-league', 'la-liga', 'serie-a',
           'ligue-1', 'english-championship', 'bundesliga-2', 'eredivisie',
           'premier-league', 'premiership', 'primeira-liga', 'super-league',
           'first-division-a', 'super-lig', 'ekstraklasa', 'one']

seasons = ['2000-2001', '2001-2002', '2002-2003', '2003-2004',
           '2004-2005', '2005-2006', '2006-2007', '2007-2008',
           '2008-2009', '2009-2010', '2010-2011', '2011-2012',
           '2012-2013', '2013-2014', '2014-2015', '2015-2016',
           '2016-2017', '2017-2018']

## Merge

+ Get fduk csvs - cll this merged_csv (still within season)
+ Get Image Paths
+ Write image refs to merged_csv
+ save new csv and copy images based on relative address in merged_csv
+ Get odds data
+ Merge into merged_csv
+ Write merged_csv

In [3]:
def form_fdcu_fps(CLEANED_DATA_DIR, nation, league, seasons):
    """
    Accepts directory names to enable reach into raw data directory
    Returns full filepaths of the data files
    This is different - different signature no source - to other versions - needs to be standardized
    """
    fps = []
    fdcu_specifics = ['football-data-co-uk', 'season-data']
    for season in seasons:
        fn = season + '.csv'
        stub = CLEANED_DATA_DIR / fdcu_specifics[0] / nation / league / season
        fp = stub / fn
        #if fp.is_file():
        if fp.exists():
            fps.append(fp)
    return fps

fps = form_fdcu_fps(STDZED_DATA_DIR, nations[0], leagues[0], seasons)

In [4]:
all_fps = []
for nation, league in zip(nations, leagues):
    fps = form_fdcu_fps(STDZED_DATA_DIR, nation, league, seasons)
    all_fps.extend(fps)

print(len(all_fps))

13


In [5]:
season_dfs = []
for fp in all_fps:
    season_df = pd.read_csv(fp, parse_dates=['date'], index_col=None)
    season_dfs.append(season_df)
season_dfs[0].head()

,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_yellowCards,a,odds_awin_bet365,odds_draw_bet365,odds_hwin_bet365,odds_awin_BW,odds_draw_BW,odds_hwin_BW,n_Bb1X2,n_BbAsian,BbAsian_handicap,odds_ftgoalsu2.5_bbmean,odds_ftgoalso2.5_bbmean,odds_awin_bbmean,odds_asianaway_bbmean,odds_asianhome_bbmean,odds_draw_bbmean,odds_hwin_bbmean,odds_ftgoalsu2.5_bbmax,odds_ftgoalso2.5_bbmax,odds_awin_bbmax,odds_asianaway_bbmax,odds_asianhome_bbmax,odds_draw_bbmax,odds_hwin_bbmax,n_BbOU,date,a_ftGoals,h_ftGoals,ftResult,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,a_htGoals,h_htGoals,h_yellowCards,h,odds_awin_IW,odds_draw_IW,odds_hwin_IW,odds_awin_LB,odds_draw_LB,odds_hwin_LB,odds_awin_VC,odds_draw_VC,odds_hwin_VC,odds_awin_WH,odds_draw_WH,odds_hwin_WH,league,nation,season,clodds_away_pinn,clodds_draw_pinn,clodds_hwin_pinn
0,8.0,16.0,0.0,13.0,6.0,2.0,bolton-wanderers,3.00,3.25,2.30,3.15,3.25,2.10,56.0,22.0,-0.25,1.70,2.01,3.05,1.84,2.01,3.16,2.20,1.87,2.20,3.40,1.92,2.10,3.30,2.40,36.0,2005-08-13,2.0,2.0,D,7.0,14.0,0.0,3.0,2.0,2.0,2.0,0.0,aston-villa,3.1,3.0,2.1,3.20,3.00,2.10,3.10,3.25,2.20,2.80,3.2,2.20,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN
1,6.0,14.0,0.0,12.0,5.0,1.0,manchester-united,1.72,3.40,5.00,1.75,3.35,4.35,56.0,23.0,0.75,1.79,1.93,1.69,1.86,2.00,3.36,4.69,1.87,2.10,1.80,1.93,2.05,3.70,5.65,36.0,2005-08-13,2.0,0.0,A,8.0,15.0,0.0,10.0,5.0,1.0,0.0,3.0,everton,1.8,3.1,3.8,1.83,3.20,3.75,1.80,3.30,4.50,1.72,3.2,4.33,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN
2,6.0,13.0,0.0,7.0,4.0,2.0,birmingham-city,2.87,3.25,2.37,2.80,3.20,2.30,56.0,21.0,0.00,1.69,2.04,2.87,2.05,1.81,3.16,2.31,1.77,2.24,3.05,2.11,1.85,3.30,2.60,36.0,2005-08-13,0.0,0.0,D,6.0,12.0,0.0,15.0,7.0,0.0,0.0,1.0,fulham,2.9,3.0,2.2,2.88,3.00,2.25,2.80,3.25,2.35,2.62,3.2,2.30,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN
3,6.0,11.0,0.0,13.0,3.0,3.0,west-bromwich-albion,5.00,3.40,1.72,4.80,3.45,1.65,55.0,23.0,-0.75,1.77,1.94,4.79,1.76,2.10,3.38,1.69,1.90,2.10,5.60,1.83,2.19,3.63,1.80,36.0,2005-08-13,0.0,0.0,D,3.0,13.0,0.0,15.0,8.0,0.0,0.0,2.0,manchester-city,4.2,3.2,1.7,4.50,3.25,1.67,5.00,3.25,1.75,4.33,3.3,1.70,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN
4,0.0,11.0,0.0,16.0,7.0,3.0,liverpool,2.40,3.20,2.87,2.20,3.35,2.90,56.0,21.0,0.25,1.71,2.02,2.35,2.00,1.86,3.17,2.81,1.80,2.23,2.50,2.04,1.89,3.35,3.30,36.0,2005-08-13,0.0,0.0,D,5.0,17.0,1.0,4.0,2.0,0.0,0.0,2.0,middlesbrough,2.5,3.0,2.5,2.30,3.10,2.75,2.35,3.25,2.80,2.30,3.1,2.75,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN


In [6]:
# create relative image paths based on standard format
for season_df in season_dfs:
    season_df.loc[:, 'shotmap_path'] = 'shotmaps' + '/' + season_df['season'] + '__' \
                                + season_df['date'].dt.strftime('%Y-%m-%d') \
                                +   '__' + season_df['h'] + '__' + season_df['a'] + '.png'
    season_df.loc[:, 'heatmap_path'] = 'heatmaps' + '/' + season_df['season'] + '__' \
                                + season_df['date'].dt.strftime('%Y-%m-%d') \
                                +   '__' + season_df['h'] + '__' + season_df['a'] + '.png'

In [7]:
season_dfs[0].head()

,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_yellowCards,a,odds_awin_bet365,odds_draw_bet365,odds_hwin_bet365,odds_awin_BW,odds_draw_BW,odds_hwin_BW,n_Bb1X2,n_BbAsian,BbAsian_handicap,odds_ftgoalsu2.5_bbmean,odds_ftgoalso2.5_bbmean,odds_awin_bbmean,odds_asianaway_bbmean,odds_asianhome_bbmean,odds_draw_bbmean,odds_hwin_bbmean,odds_ftgoalsu2.5_bbmax,odds_ftgoalso2.5_bbmax,odds_awin_bbmax,odds_asianaway_bbmax,odds_asianhome_bbmax,odds_draw_bbmax,odds_hwin_bbmax,n_BbOU,date,a_ftGoals,h_ftGoals,ftResult,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,a_htGoals,h_htGoals,h_yellowCards,h,odds_awin_IW,odds_draw_IW,odds_hwin_IW,odds_awin_LB,odds_draw_LB,odds_hwin_LB,odds_awin_VC,odds_draw_VC,odds_hwin_VC,odds_awin_WH,odds_draw_WH,odds_hwin_WH,league,nation,season,clodds_away_pinn,clodds_draw_pinn,clodds_hwin_pinn,shotmap_path,heatmap_path
0,8.0,16.0,0.0,13.0,6.0,2.0,bolton-wanderers,3.00,3.25,2.30,3.15,3.25,2.10,56.0,22.0,-0.25,1.70,2.01,3.05,1.84,2.01,3.16,2.20,1.87,2.20,3.40,1.92,2.10,3.30,2.40,36.0,2005-08-13,2.0,2.0,D,7.0,14.0,0.0,3.0,2.0,2.0,2.0,0.0,aston-villa,3.1,3.0,2.1,3.20,3.00,2.10,3.10,3.25,2.20,2.80,3.2,2.20,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN,shotmaps/2005-2006__2005-08-13__aston-villa__b...,heatmaps/2005-2006__2005-08-13__aston-villa__b...
1,6.0,14.0,0.0,12.0,5.0,1.0,manchester-united,1.72,3.40,5.00,1.75,3.35,4.35,56.0,23.0,0.75,1.79,1.93,1.69,1.86,2.00,3.36,4.69,1.87,2.10,1.80,1.93,2.05,3.70,5.65,36.0,2005-08-13,2.0,0.0,A,8.0,15.0,0.0,10.0,5.0,1.0,0.0,3.0,everton,1.8,3.1,3.8,1.83,3.20,3.75,1.80,3.30,4.50,1.72,3.2,4.33,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN,shotmaps/2005-2006__2005-08-13__everton__manch...,heatmaps/2005-2006__2005-08-13__everton__manch...
2,6.0,13.0,0.0,7.0,4.0,2.0,birmingham-city,2.87,3.25,2.37,2.80,3.20,2.30,56.0,21.0,0.00,1.69,2.04,2.87,2.05,1.81,3.16,2.31,1.77,2.24,3.05,2.11,1.85,3.30,2.60,36.0,2005-08-13,0.0,0.0,D,6.0,12.0,0.0,15.0,7.0,0.0,0.0,1.0,fulham,2.9,3.0,2.2,2.88,3.00,2.25,2.80,3.25,2.35,2.62,3.2,2.30,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN,shotmaps/2005-2006__2005-08-13__fulham__birmin...,heatmaps/2005-2006__2005-08-13__fulham__birmin...
3,6.0,11.0,0.0,13.0,3.0,3.0,west-bromwich-albion,5.00,3.40,1.72,4.80,3.45,1.65,55.0,23.0,-0.75,1.77,1.94,4.79,1.76,2.10,3.38,1.69,1.90,2.10,5.60,1.83,2.19,3.63,1.80,36.0,2005-08-13,0.0,0.0,D,3.0,13.0,0.0,15.0,8.0,0.0,0.0,2.0,manchester-city,4.2,3.2,1.7,4.50,3.25,1.67,5.00,3.25,1.75,4.33,3.3,1.70,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN,shotmaps/2005-2006__2005-08-13__manchester-cit...,heatmaps/2005-2006__2005-08-13__manchester-cit...
4,0.0,11.0,0.0,16.0,7.0,3.0,liverpool,2.40,3.20,2.87,2.20,3.35,2.90,56.0,21.0,0.25,1.71,2.02,2.35,2.00,1.86,3.17,2.81,1.80,2.23,2.50,2.04,1.89,3.35,3.30,36.0,2005-08-13,0.0,0.0,D,5.0,17.0,1.0,4.0,2.0,0.0,0.0,2.0,middlesbrough,2.5,3.0,2.5,2.30,3.10,2.75,2.35,3.25,2.80,2.30,3.1,2.75,english-premier-league,united-kingdom,2005-2006,NaN,NaN,NaN,shotmaps/2005-2006__2005-08-13__middlesbrough_...,heatmaps/2005-2006__2005-08-13__middlesbrough_...


In [8]:
def copy_data(src_fps, dest_fps):
    """
    Accepts a list of strings representing filepaths
    Copies the files if they do not exist, otherwise counts if already there
    Returns the number of files copied, and number of files already existing
    """
#     dest_fps = []
#     for src_fp in src_fps:
#         #dest_fps.append(src_fp.replace(str(env_dir), str(project_dir)))
    n_copied = 0
    n_exist = 0
    for src_fp, dest_fp in zip(src_fps, dest_fps):
#         src_fp = pathlib.Path(src_fp)
#         dest_fp = pathlib.Path(dest_fp)
        if src_fp.exists():
            if not dest_fp.exists():
                dest_fp.parent.mkdir(parents=True, exist_ok=True)
                # Copy the original files without touching them
                dest_fp.write_bytes(src_fp.read_bytes())
                n_copied += 1
            else:
                n_exist += 1
    return n_copied, n_exist

In [9]:
# Create the full source and destination paths
for season_df in season_dfs:
    rel_path = season_df['shotmap_path']
    stub = None
    dest_fps = season_df['nation'] + '/' + season_df['league'] + '/' + season_df['season'] + '/' + rel_path
    full_dest_fps = [pathlib.Path(str(MERGED_DATA_DIR) + '/' + fp) for fp in dest_fps]
    src_fps = [str(STDZED_DATA_DIR) + '/' + 'whoscored-com' + '/' + fp for fp in dest_fps]
    final_src_fps = [pathlib.Path(fp.replace(fp.split('__')[1], 'date')) for fp in src_fps]
    # Copy the files
    copy_data(final_src_fps, full_dest_fps)
print(final_src_fps[-1])
print('\n')
print(full_dest_fps[-1])

/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-standardized/whoscored-com/united-kingdom/english-premier-league/2017-2018/shotmaps/2017-2018__date__west-ham-united__everton.png


/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/04-merged/united-kingdom/english-premier-league/2017-2018/shotmaps/2017-2018__2018-05-13__west-ham-united__everton.png


In [10]:
# Create the full source and destination paths
for season_df in season_dfs:
    rel_path = season_df['heatmap_path']
    stub = None
    dest_fps = season_df['nation'] + '/' + season_df['league'] + '/' + season_df['season'] + '/' + rel_path
    full_dest_fps = [pathlib.Path(str(MERGED_DATA_DIR) + '/' + fp) for fp in dest_fps]
    src_fps = [str(STDZED_DATA_DIR) + '/' + 'whoscored-com' + '/' + fp for fp in dest_fps]
    final_src_fps = [pathlib.Path(fp.replace(fp.split('__')[1], 'date')) for fp in src_fps]
    # Copy the files
    copy_data(final_src_fps, full_dest_fps)
print(final_src_fps[-1])
print('\n')
print(full_dest_fps[-1])

/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-standardized/whoscored-com/united-kingdom/english-premier-league/2017-2018/heatmaps/2017-2018__date__west-ham-united__everton.png


/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/04-merged/united-kingdom/english-premier-league/2017-2018/heatmaps/2017-2018__2018-05-13__west-ham-united__everton.png


### Get odds data

In [11]:
def form_ic_fps(STDZED_DATA_DIR, nation, league, seasons):
    """
    Accepts directory names to enable reach into raw data directory
    Returns full filepaths of the data files
    This is different - different signature no source - to other versions - needs to be standardized
    """
    fps = []
    fdcu_specifics = ['indatabet-com', 'season-data']
    for season in seasons:
        fn = season + '.csv'
        stub = STDZED_DATA_DIR / fdcu_specifics[0] / nation / league / season
        fp = stub / fn
        #if fp.is_file():
        if fp.exists():
            fps.append(fp)
    return fps

fps = form_ic_fps(STDZED_DATA_DIR, nations[0], leagues[0], seasons)

In [12]:
all_odds_fps = []
for nation, league in zip(nations, leagues):
    fps = form_ic_fps(STDZED_DATA_DIR, nation, league, seasons)
    all_odds_fps.extend(fps)

print(len(all_odds_fps))
print(all_odds_fps[0])

11
/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-standardized/indatabet-com/united-kingdom/english-premier-league/2007-2008/2007-2008.csv


In [13]:
odds_dfs = []
for fp in all_odds_fps:
    odds_df = pd.read_csv(fp, parse_dates=['date'], index_col=None)
    odds_dfs.append(odds_df)
odds_dfs[0].head()

,date,id_fifa,nation,league,season,h,a,h_ftGoals,a_ftGoals,odds_hwin_pinn,odds_draw_pinn,odds_awin_pinn,odds_hwin_bet365,odds_draw_bet365,odds_awin_bet365
0,2007-08-11,eng-pl,united-kingdom,english-premier-league,2007-2008,sunderland,tottenham-hotspur,1.0,0.0,3.40,3.35,2.20,2.80,3.25,2.25
1,2007-08-11,eng-pl,united-kingdom,english-premier-league,2007-2008,west-ham-united,manchester-city,0.0,2.0,2.16,3.25,3.60,1.83,3.20,4.00
2,2007-08-11,eng-pl,united-kingdom,english-premier-league,2007-2008,middlesbrough,blackburn-rovers,1.0,2.0,2.57,3.20,2.95,2.25,3.25,2.80
3,2007-08-11,eng-pl,united-kingdom,english-premier-league,2007-2008,everton,wigan-athletic,2.0,1.0,1.73,3.50,5.40,1.62,3.50,5.00
4,2007-08-11,eng-pl,united-kingdom,english-premier-league,2007-2008,derby,portsmouth,2.0,2.0,3.20,3.20,2.35,2.50,3.25,2.50


## Merge

In [14]:
def merger(football_df, odds_df2):
    #dfs = sorted(list(df_dict.values()), key=lambda x: len(x), reverse=True)
    # merge_asof on date using home team, awat_team, home_goals, and away_goals to match
    # merge_asof does a left join, so put longest daf on left, so get max data into merged
    merged = pd.merge_asof(football_df, odds_df2,
                           on='date',
                           by=['h', 'a', 'h_ftGoals', 'a_ftGoals'],
                           suffixes=('_ic', '_fdcu'),
                           tolerance=pd.Timedelta(days=2),
                           direction='nearest'
                           )
    # # Put a date difference column into the merged df
    # merged['dates_diff'] = merged['date_ic'] - merged['date_fdcu']
    # # Write the merge issues data to a yaml file

    merged.sort_values(by='date', ascending=True, inplace=True)
    return merged

In [15]:
for df in season_dfs:
    df.sort_values(by=['date'], inplace=True)
    season = df['season'].unique()[0]
    nation = df['nation'].unique()[0]
    league = df['league'].unique()[0]
    # Does the odds df exist?
    fn = season + '.csv'
    fp = STDZED_DATA_DIR / 'indatabet-com' / nation / league / season / fn
    if fp.exists():
#         print(fp)
#         print(df.head())
        dest_fp = MERGED_DATA_DIR / nation / league / season / fn
        if not dest_fp.exists():
            dest_fp.parent.mkdir(parents=True, exist_ok=True)            
        odds_df = pd.read_csv(fp, parse_dates=['date'], index_col=None)
        odds_df.sort_values(by=['date'], inplace=True)
        #print(odds_df.head())
        # print(merged_df.head())
        merged_df = merger(df, odds_df)
        merged_df.to_csv(dest_fp, index=None)        #df.head()
#         print(merged_df.head())
    
    
#     break
    
    # print(df.head())